## News :D



In [2]:
%load_ext autoreload
%autoreload 2

import requests
import re 
import itertools
from bs4 import BeautifulSoup # Imports bs4
import wget # Imports wget

# Data

### Data formats

CNN has links by month

NYT has links by day

BI has links by month

TH has links by day

NYP has links by day

DM has links by day

Fox has links by page num

### Thoughts on how to organize the data 

we use a pandas dataframe organized like this

| Dates | CNN | NYT | BI | TH | NYP | DM | FOX |
| ----------- | ------------------------------------------ | ------- | ------- | ------- | ------- | ------- | ------- |
| 2014-01-01  | list of tuples with article link & title   | ... | ... | ... | ... | ... | ... |
| 2014-01-02  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-03  | ... | ... | ... | ... | ... | ... | ... |
| 2014-01-04  | ... | ... | ... | ... | ... | ... | ... |
| ...         | ... | ... | ... | ... | ... | ... | ... |
| 2024-12-31  | ... | ... | ... | ... | ... | ... | ... |

### Code to get data
CNN:
```html
<li>
    <span class="date">2024-01-31</span>
    <span class="sitemap-link">
        <a href="https://www.cnn.com/tech/live-news/meta-x-discord-tiktok-snap-chiefs-testimony-senate/index.html">
            Mark Zuckerberg apologizes to families over social media harms in contentious Senate hearing
        </a>
    </span>
</li>
```

NYT:
```html
<li>
    <a href="https://www.nytimes.com/2023/12/31/business/dealbook/pga-tour-saudi-deal-deadline.html">
        PGA Tour and Saudi-Backed LIV Extend Deadline to Finalize Deal
    </a>
</li>
```

BI:
```html
<p>
    <a href="https://www.businessinsider.com/feud-capote-vs-swans-ann-woodward-suicide-death-real-story-2024-1">
        The real story behind the suicide of American socialite Ann Woodward in &#39;Feud: Capote vs. The Swans&#39;
    </a>
    <br>2024-02-01T03:31:01.68Z
</p>
```

NYP:
```html
<h3 class="story__headline headline headline--archive">
    <a href="https://nypost.com/2024/01/01/sports/og-anunoby-trending-toward-the-perfect-piece-for-knicks/" rel="" target="_self">
		Knicks&#039; big trade acquisition already looks like the &#039;perfect fit&#039; 												
    </a>
</h3>
```

DM:
```html
<a href="/wires/ap/article-12917573/Christian-McCaffrey-miss-49ers-regular-season-finale-return-playoffs.html" title="">
    Christian McCaffrey will miss the 49ers regular-season finale but should return for playoffs
</a>
```

FOX:
```html
<loc>
    https://www.foxnews.com/opinion/why-the-department-of-education-is-going-to-rip
</loc>
<lastmod>
    2025-03-12T05:00:54-04:00
</lastmod>
```

In [3]:
months = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]
mday = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# CNN
cnn_dict = dict(
    [
        (
            i,
            [
                f"https://www.cnn.com/article/sitemap-{i}-{j}.html"
                for j in range(1, 13)
            ],
        )
        for i in range(2014, 2025)
    ]
)


# New York Times
nyt_dict = dict(
    [
        (
            i,
            dict(
                [
                    (
                        j,
                        [
                            f"https://www.nytimes.com/sitemap/{i}/{str(j).zfill(2)}/{str(k).zfill(2)}/"
                            for k in range(1, mday[j - 1]+1)
                        ],
                    )
                    for j in range(1, 13)
                ]
            ),
        )
        for i in range(2014, 2025)
    ]
)


# Business Insider
bi_dict = dict(
    [
        (
            i,
            [
                f"https://www.businessinsider.com/sitemap/html/{i}-{str(j).zfill(2)}.html"
                for j in range(1, 13)
            ],
        )
        for i in range(2014, 2025)
    ]
)


# New York Post
nyp_dict = dict(
    [
        (
            i,
            dict(
                [
                    (
                        j,
                        [
                            f"https://nypost.com/{i}/{str(j).zfill(2)}/{str(k).zfill(2)}/"
                            for k in range(1, mday[j - 1]+1)
                        ],
                    )
                    for j in range(1, 13)
                ]
            ),
        )
        for i in range(2014, 2025)
    ]
)


# Daily Mail
dm_dict = dict(
    [
        (
            i,
            dict(
                [
                    (
                        j,
                        [
                            f"https://www.dailymail.co.uk/home/sitemaparchive/day_{i}{str(j).zfill(2)}{str(k).zfill(2)}.html"
                            for k in range(1, mday[j - 1]+1)
                        ],
                    )
                    for j in range(1, 13)
                ]
            ),
        )
        for i in range(2014, 2025)
    ]
)


# Fox
fox_links = ["https://www.foxnews.com/sitemap.xml?type=articles"] + [
    f"https://www.foxnews.com/sitemap.xml?type=articles&page={i}"
    for i in range(2, 161)
]

In [ ]:
# Everything Below this was used to test getting articles names, links, and dates from scraped data
# not all final



r = requests.get(fox_links[0])
print('discovery' in r.text)


cnn_articles = {}
for year in range(2014,2025):
    for month in range(1,13):

        r = requests.get(cnn_dict[year][month])
        soup = BeautifulSoup(r.text, "html.parser")
        test = [i for i in soup.find_all("li")]

        titles = []
        links = []
        dates = []

        for i in test:
            for j in i.descendants:
                if "sitemap-link" in str(j):
                    titles.append(
                        str(j)[
                            36
                            + str(j)[36:].index('"')
                            + 2 : 36
                            + str(j)[36:].index("<")
                        ]
                    )
                    links.append(str(j)[36 : 36 + str(j)[36:].index('"')])

print(titles)
print(links)
print(dates)

# r = requests.get(nyt_dict[2024][1][0])
# soup = BeautifulSoup(r.text, "html.parser")
# # print(r.text)
# test = [i for i in soup.find_all("li")]
# titles = []
# links = []
# dates = []
# for i in test:
#     test = str(i.contents[0])
#     if ("https:" in test) & (".html" in test) & ("2024/01/01" in test):
#         links.append(test[test.index("https:") : test.index(".html")])
#         titles.append(test[test.index(">") + 1 : test[4:].index("<") + 4])
#         dates.append("2024-01-01")

# print(titles)
# print(links)
# print(dates)

True


In [8]:
# Pandas Testing area
import pandas as pd

test_frame = pd.DataFrame()